# Contextual Chunk Headers (CCH) in Simple RAG

### CCH 활용한 RAG 파이프라인

**1. 데이터 수집 및 전처리:**
PDF, 웹 문서, 내부 보고서 등에서 텍스트 데이터를 수집하고 정리한다.
이때 문서 구조(예: 제목, 섹션, 부제목 등)를 유지한 채 파싱하는 것이 중요하다.

**2. 문맥 헤더를 사용한 청크 구성:**
각 청크에 해당 섹션의 제목이나 상위 헤더 정보를 프리픽스(prefix)로 추가한다.
예: " 3.2 제품 보안 정책\n본 제품은 사용자 데이터를 암호화하여…"
이는 각 청크가 어떤 주제를 다루는지를 명확히 하여 검색 정확도를 높인다.

**3. 임베딩 생성:**
문맥이 강화된 청크를 임베딩 모델을 사용해 벡터화한다.
벡터는 벡터 데이터베이스에 저장되어 이후 검색에 활용된다.

**4. 시맨틱 검색 (Semantic Search):**
사용자 쿼리에 대해 유사도가 높은 문맥 강화 청크를 검색한다.
이 과정에서는 본문 내용과 함께 헤더의 의미도 반영되어 더 정교한 검색 결과가 도출된다.

**5. 응답 생성:**
검색된 청크들을 바탕으로 언어 모델이 최종 응답을 생성한다.
문맥에 맞는 내용만 활용되기 때문에, 논리적 일관성과 사실 정확도가 향상된다.

**6. 평가:**
응답 결과에 대해 정확도, 충실도, 관련성 등을 기준으로 평가한다.
평가 도구로는 자동 점수 지표(ROUGE, BLEU 등)나 GPT 기반 평가, 인적 평가가 사용된다.

### Contextual Chunk Headers(CCH)의 효과

- 검색 품질 향상: 상위 문맥 정보를 활용하여 의미 기반 검색의 정확도를 향상시킨다.
- 문맥 일관성 유지: 생성된 응답이 본래의 문서 구조와 일치하므로, 주제에 벗어난 결과를 줄일 수 있다.
- 도메인 특화 응답에 효과적: 기술 문서, 정책 매뉴얼 등 섹션 구성이 명확한 자료에서 특히 유리하다.

## Setting Up the Environment

In [1]:
import numpy as np
import json
import fitz
from tqdm import tqdm

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text and Identifying Section Headers

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출하고 첫 번째 `num_chars` 문자를 인쇄합니다.

    Args:
    PDF_path(str): PDF 파일의 경로입니다.

    Returns:
    str: PDF에서 추출한 텍스트.
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 빈 문자열을 초기화합니다.

    # PDF의 각 페이지를 반복합니다.
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  
        text = page.get_text("text")  
        # 추출한 텍스트를 all_text 문자열에 추가합니다.
        all_text += text  

    return all_text

## Chunking Text with Contextual Headers

In [4]:
def generate_chunk_header(chunk, model="gpt-4o-mini"):
    """
    LLM을 사용하여 지정된 텍스트 청크의 제목/헤더를 생성합니다.

    Args:
    chunk (str): 헤더로 요약할 텍스트 청크입니다.
    model (str): 헤더를 생성하는 데 사용할 모델입니다. 기본값은 "gpt-4o-mini"입니다.

    Returns:
    str: 생성된 헤더/제목.
    """
    
    # AI의 동작을 안내하는 시스템 프롬프트 정의하기
    system_prompt = "Generate a concise and informative title for the given text."
    
    # 시스템 프롬프트 및 텍스트 청크를 기반으로 AI 모델에서 응답을 생성합니다.
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": chunk}
        ]
    )
    # 생성된 헤더/제목을 반환하고 선행/후행 strip 함수로 공백을 제거합니다.
    return response.choices[0].message.content.strip()

In [5]:
def chunk_text_with_headers(text, n, overlap):

    """
    청크 텍스트를 더 작은 세그먼트로 나누고 헤더를 생성합니다.

    Args:
    text (str): 청크할 전체 텍스트입니다.
    n (int): 문자 단위의 청크 크기입니다.
    overlap (int): 청크 사이의 겹치는 문자 수입니다.

    Returns:
    List[dict]: 'header'와 'text' 키를 가진 딕셔너리 리스트입니다.
    """
    
    chunks = []  # 청크를 저장하기 위해 공백 리스트 지정

    # 지정된 청크 크기와 겹침으로 텍스트를 반복합니다.
    for i in range(0, len(text), n - overlap):
        chunk = text[i:i + n]  # 텍스트 청크 추출
        header = generate_chunk_header(chunk)  # LLM을 사용하여 청크의 헤더를 생성합니다.
        chunks.append({"header": header, "text": chunk})  # 목록에 헤더와 청크를 추가합니다.

    return chunks  # 헤더가 있는 청크 목록을 반환합니다.

## Extracting and Chunking Text from a PDF File

In [6]:
# PDF 파일 경로 정의
pdf_path = "../../dataset/AI_Understanding.pdf"

# PDF 파일에서 텍스트 추출
extracted_text = extract_text_from_pdf(pdf_path)

# 추출된 텍스트를 헤더로 청크화합니다.
# 청크 크기는 1000자, 겹침은 200자를 사용합니다.
text_chunks = chunk_text_with_headers(extracted_text, 1000, 200)

# 생성된 헤더와 함께 샘플 청크를 인쇄합니다.
print("Sample Chunk:")
print("Header:", text_chunks[0]['header'])
print("Content:", text_chunks[0]['text'])

Sample Chunk:
Header: "인공 지능의 이해: 역사, 현대적 응용 및 핵심 개념"
Content: 인공 지능 이해 
 
1장: 인공 지능 소개 
인공 지능(AI)은 디지털 컴퓨터 또는 컴퓨터로 제어되는 로봇이 지적인 존재와 일반적으로 
관련된 작업을 수행할 수 있는 능력을 말합니다. 이 용어는 추론, 의미 발견, 일반화, 과거 
경험으로부터의 학습 능력 등 인간의 특징적인 지적 프로세스가 부여된 시스템을 
개발하는 프로젝트에 자주 적용됩니다. 지난 수십 년 동안 컴퓨팅 성능과 데이터 가용성의 
발전으로 AI의 개발과 배포가 크게 가속화되었습니다. 
역사적 맥락 
인공 지능에 대한 개념은 수세기 동안 존재해 왔으며 종종 신화와 소설에 묘사되기도 
했습니다. 하지만 공식적인 AI 연구 분야는 20세기 중반에 시작되었습니다. 1956년 
다트머스 워크숍은 AI의 발상지로 널리 알려져 있습니다. 초기 AI 연구는 문제 해결과 
상징적 방법에 중점을 두었습니다. 1980년대에는 전문가 시스템이 등장했고, 1990년대와 
2000년대에는 머신러닝과 신경망이 발전했습니다. 최근 딥러닝의 획기적인 발전은 이 
분야에 혁신을 가져왔습니다. 
현대 관측 
최신 AI 시스템은 일상 생활에서 점점 더 널리 보급되고 있습니다. Siri와 Alexa 같은 가상 
비서부터 스트리밍 서비스 및 소셜 미디어의 추천 알고리즘에 이르기까지 AI는 우리의 
생활, 업무, 상호 작용 방식에 영향을 미치고 있습니다. 자율 주행 자동차, 첨단 의료 진단, 
정교한 재무 모델링 도구의 개발은 AI가 광범위하고 성장하는 응용 분야를 보여줍니다. 
윤리적 영향, 편견, 일자리 대체에 대한 우려도 점점 더 커지고 있습니다. 
 
2장: 인공 지능의 핵심 개념 
머신 러닝 
머신러닝(ML)은 명시적으로 프로그래밍하지 않고도 시스템이 데이터로부터 학습할 수 
있도록 하는 데 중점을 둔 AI의 하위 집합입니다. ML 알고리즘은 더 많은 데이터에 
노출됨에 따라 패턴을 식별하고 예측하며 시간이 지남에

## Creating Embeddings for Headers and Text

In [7]:
def create_embeddings(text, model="text-embedding-3-small"):
    
    """
    주어진 텍스트에 대한 임베딩을 생성합니다.

    Args:
    text (str): 임베드할 입력 텍스트입니다.
    model (str): 사용할 임베딩 모델입니다. 기본값은 "text-embedding-3-small"입니다.

    Returns:
    dict: 입력 텍스트에 대한 임베딩이 포함된 응답입니다.
    """
    # 지정된 모델과 입력 텍스트를 사용하여 임베딩을 만듭니다.
    response = client.embeddings.create(
        model=model,
        input=text
    )
    # 응답에서 임베딩을 반환합니다.
    return response.data[0].embedding

In [8]:
# 각 청크에 대한 임베딩 생성하기
embeddings = [] 

# 진행률 표시줄을 사용하여 각 텍스트 청크를 반복합니다.
for chunk in tqdm(text_chunks, desc="Generating embeddings"):
    # 청크의 텍스트에 대한 임베딩 만들기
    text_embedding = create_embeddings(chunk["text"])
    # 청크의 헤더에 임베딩을 생성합니다.
    header_embedding = create_embeddings(chunk["header"])
    # 청크의 헤더, 텍스트 및 해당 임베딩을 목록에 추가합니다.
    embeddings.append({"header": chunk["header"], "text": chunk["text"], "embedding": text_embedding, "header_embedding": header_embedding})

Generating embeddings: 100%|██████████| 21/21 [00:23<00:00,  1.14s/it]


## Performing Semantic Search

In [9]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 사이의 코사인 유사도를 계산합니다.

    Args:
    vec1 np.ndarray)): 첫 번째 벡터.
    vec2 (np.ndarray): 두 번째 벡터.

    Returns:
    float: 코사인 유사도 점수.
    """

    # 두 벡터의 내적 곱을 계산합니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [10]:
def semantic_search(query, chunks, k=5):
    """
    쿼리를 기반으로 가장 관련성이 높은 청크를 검색합니다.

    Args:
    query (str): 사용자 쿼리.
    chunk (List[딕셔너리]): 임베딩이 있는 텍스트 청크의 목록.
    k (int): 상위 결과의 개수.

    Returns:
    List[dict]: 가장 관련성이 높은 상위 k개 청크.
    """
    # 쿼리에 대한 임베딩을 생성합니다.
    query_embedding = create_embeddings(query)

    similarities = [] # 유사성 점수를 저장할 목록을 초기화합니다.

    # 각 청크를 반복하여 유사도 점수를 계산합니다.
    for chunk in chunks:
        # 쿼리 임베딩과 청크 텍스트 임베딩 간의 코사인 유사도 계산: # 코사인 유사도 계산
        sim_text = cosine_similarity(np.array(query_embedding), np.array(chunk["embedding"]))
        # 쿼리 임베딩과 청크 헤더 임베딩 간의 코사인 유사성 계산
        sim_header = cosine_similarity(np.array(query_embedding), np.array(chunk["header_embedding"]))
        # 평균 유사도 점수 계산
        avg_similarity = (sim_text + sim_header) / 2
        # 청크와 평균 유사도 점수를 목록에 추가합니다.
        similarities.append((chunk, avg_similarity))

    # 유사도 점수를 기준으로 청크를 내림차순으로 정렬합니다.
    similarities.sort(key=lambda x: x[1], reverse=True)
    # 가장 관련성이 높은 상위 k개의 청크를 반환합니다.
    return [x[0] for x in similarities[:k]]

## Running a Query on Extracted Chunks

In [11]:
with open('dataset/validation.json') as f:
    data = json.load(f)

query = data[0]['question']

# 가장 연관성이 높은 상위 2개 텍스트 청크 검색
top_chunks = semantic_search(query, embeddings, k=2)

print("Query:", query)
for i, chunk in enumerate(top_chunks):
    print(f"Header {i+1}: {chunk['header']}")
    print(f"Content:\n{chunk['text']}\n")

Query: '설명 가능한 AI(Explainable AI)'란 무엇이며, 왜 중요한가?
Header 1: "AI 시스템의 신뢰 구축을 위한 윤리적 지침: 편향성 해결, 투명성, 개인정보 보호 및 책임"
Content:
고 사회에 유익한 AI 시스템의 개발과 
배포를 보장하기 위한 지침입니다. 주요 원칙에는 인권 존중, 개인정보 보호, 비차별, 
공익성이 포함됩니다. 
AI의 편향성 해결 
AI 시스템은 데이터에 존재하는 편견을 유전하고 증폭시켜 불공정하거나 차별적인 결과를 
초래할 수 있습니다. 편향성을 해결하려면 신중한 데이터 수집, 알고리즘 설계, 지속적인 
모니터링 및 평가가 필요합니다. 
투명성 및 설명 가능성 
투명성과 설명 가능성은 AI 시스템에 대한 신뢰를 구축하는 데 필수적입니다. 설명 가능한 
AI(XAI) 기술은 AI의 결정을 더 이해하기 쉽게 만들어 사용자가 공정성과 정확성을 
평가할 수 있도록 하는 것을 목표로 합니다. 
개인정보 및 데이터 보호 
AI 시스템은 대량의 데이터에 의존하는 경우가 많기 때문에 개인정보 보호와 데이터 
보호에 대한 우려가 제기됩니다. 책임감 있는 데이터 처리, 개인정보 보호 기술 구현, 
데이터 보호 규정 준수는 매우 중요합니다. 
책임과 의무 
AI 시스템에 대한 책임과 의무를 확립하는 것은 잠재적인 피해를 해결하고 윤리적 행동을 
보장하는 데 필수적입니다. 여기에는 AI 시스템의 개발자, 배포자, 사용자에 대한 역할과 
책임을 정의하는 것이 포함됩니다. 
 
20장: AI에 대한 신뢰 구축 
투명성 및 설명 가능성 
투명성과 설명 가능성은 AI에 대한 신뢰를 구축하는 데 있어 핵심입니다. AI 시스템을 
이해하기 쉽게 만들고 의사 결정 프로세스에 대한 인사이트를 제공하면 사용자가 AI의 
신뢰성과 공정성을 평가하는 데 도움이 됩니다. 
견고성 및 신뢰성 
AI 시스템의 견고성과 신뢰성을 확보하는 것은 신뢰를 구축하는 데 필수적입니다. 
여기에는 AI 모델 테스트 및 검증, 성능 모니터링, 잠재적인 취약점 

## Generating a Response Based on Retrieved Chunks

In [12]:
# AI 어시스턴트를 위한 시스템 프롬프트 정의
system_prompt = (
    "당신은 주어진 문맥을 기반으로만 답변하는 AI 어시스턴트입니다. "
    "제공된 문맥에서 직접적으로 답을 도출할 수 없는 경우에는 다음과 같이 응답하십시오: "
    "'I do not have enough information to answer that.'"
)

def generate_response(system_prompt, user_message, model="gpt-4o-mini"):
    """
    시스템 프롬프트와 사용자 메시지를 기반으로 AI 모델의 응답을 생성합니다.

    Args:
    system_prompt (str): AI의 행동 방식을 정의하는 시스템 메시지
    user_message (str): 사용자의 질문 또는 입력
    model (str): 사용할 언어 모델 (기본값: "meta-llama/Llama-3.2-3B-Instruct")

    Returns:
    dict: AI 모델의 응답
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,  # 일관된 응답을 위해 창의성 최소화
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# 상위 문맥 청크들을 기반으로 사용자 프롬프트 생성
# 각 청크는 딕셔너리 형식이며 'header'와 'text' 키를 가짐
user_prompt = "\n".join([
    f"제목: {chunk['header']}\n내용:\n{chunk['text']}" 
    for chunk in top_chunks
])

# 질문을 프롬프트 마지막에 추가
user_prompt = f"{user_prompt}\n질문: {query}"

# AI 응답 생성
ai_response = generate_response(system_prompt, user_prompt)

## Evaluating the AI Response

In [13]:
# 평가 시스템용 프롬프트 정의
evaluate_system_prompt = """당신은 지능형 평가 시스템입니다.
제공된 문맥을 바탕으로 AI 어시스턴트의 응답을 평가하십시오.
- 정답과 매우 유사한 경우 점수는 1점을 부여하십시오.
- 부분적으로 맞은 경우는 0.5점을 부여하십시오.
- 틀린 경우는 0점을 부여하십시오.
반환값은 오직 점수 (0, 0.5, 1) 중 하나여야 합니다."""

# 검증 데이터에서 정답 추출
true_answer = data[0]['ideal_answer']

# 평가용 프롬프트 생성
evaluation_prompt = f"""
사용자 질문: {query}
AI 응답: {ai_response.choices[0].message.content}
정답: {true_answer}
{evaluate_system_prompt}
"""

# 평가 점수 생성 (AI 모델에 평가 프롬프트 전달)
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 평가 점수 출력
print("평가 점수:", evaluation_response.choices[0].message.content)

평가 점수: 1
